In [53]:
# import relavent packages
import pandas as pd
import requests
import ftplib
import os
import ftplib
import numpy as np
print('works') 

works


In [54]:
# looks for the file online
ftp_server = 'ftp.ebi.ac.uk'
ftp_base_path = 'pub/databases/gwas/summary_statistics/GCST003001-GCST004000/GCST003659/'
accession_id = 'GCST003659'
ftp_file_name = 'MAGIC_ISI_Model_3_JMA.txt'

try:
    with ftplib.FTP(ftp_server) as ftp:
        ftp.login()
        print(f"Connected to FTP server: {ftp_server}")
        ftp.cwd(ftp_base_path)
        print(f"Changed to directory: {ftp_base_path}")
        files = ftp.nlst()
        print(f"Files in directory {ftp_base_path}: {files}")
        if ftp_file_name in files:
            print(f"File {ftp_file_name} exists in the directory.")
        else:
            print(f"File {ftp_file_name} does not exist in the directory.")
except ftplib.all_errors as e:
    print(f"FTP error: {e}")

# now to actually download the file...
local_filename = f'{accession_id}_{ftp_file_name}'

try:
    with ftplib.FTP(ftp_server) as ftp:
        ftp.login()
        print(f"Connected to FTP server: {ftp_server}")
        ftp.cwd(ftp_base_path)
        print(f"Changed to directory: {ftp_base_path}")
        with open(local_filename, 'wb') as local_file:
            ftp.retrbinary(f'RETR {ftp_file_name}', local_file.write)
    print(f"File {local_filename} downloaded successfully.")
except ftplib.all_errors as e:
    print(f"FTP error: {e}")

if os.path.exists(local_filename):
    print(f"Downloaded file {local_filename} exists.")
else:
    print(f"Downloaded file {local_filename} does not exist.")


Connected to FTP server: ftp.ebi.ac.uk
Changed to directory: pub/databases/gwas/summary_statistics/GCST003001-GCST004000/GCST003659/
Files in directory pub/databases/gwas/summary_statistics/GCST003001-GCST004000/GCST003659/: ['MAGIC ISI README.docx', 'MAGIC_ISI_Model_3_JMA.txt', 'harmonised']
File MAGIC_ISI_Model_3_JMA.txt exists in the directory.
Connected to FTP server: ftp.ebi.ac.uk
Changed to directory: pub/databases/gwas/summary_statistics/GCST003001-GCST004000/GCST003659/
File GCST003659_MAGIC_ISI_Model_3_JMA.txt downloaded successfully.
Downloaded file GCST003659_MAGIC_ISI_Model_3_JMA.txt exists.


In [55]:

# check if the file was actually downloaded, this is mostly for debugging
if os.path.exists(local_filename):
    try:
        # load the downloaded gwas summary stats file
        gwas_summary = pd.read_csv(local_filename, delimiter='\t')
        print("File loaded successfully.")
        # display the first few rows and column names for debugging purposes
        display(gwas_summary.head())
        print(gwas_summary.columns)
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
else:
    print(f"File {local_filename} not found.")




File loaded successfully.


,snp,chr_build36,pos_build36,effect_allele,other_allele,maf_hapmap_rel27,effect,stderr,pvalue
0,rs3094315,1,742429,a,g,0.165,-0.097,0.11,0.4250
1,rs3131968,1,744055,a,g,0.125,0.130,0.12,0.2233
2,rs12124819,1,766409,a,g,0.222,0.081,0.12,0.4208
3,rs2980319,1,766985,a,t,0.134,0.092,0.13,0.2842
4,rs2905062,1,774913,a,g,0.144,-0.077,0.13,0.3403


Index(['snp', 'chr_build36', 'pos_build36', 'effect_allele', 'other_allele',
       'maf_hapmap_rel27', 'effect', 'stderr', 'pvalue'],
      dtype='object')


In [56]:
# load gwas summary stats and genotype data from poscontrol.sh file
gwas_summary = pd.read_csv('poscontrol.sh', delim_whitespace=True)

# shows gwas stats for debugging purposes
print("GWAS Summary Statistics:")
print(gwas_summary.head())

# normalize snps in gwas summary stats
gwas_summary['snp'] = gwas_summary['snp'].str.strip().str.lower()

# making the dataframe
genotype_df = gwas_summary[['snp', 'chr_build36', 'pos_build36', 'effect_allele', 'other_allele', 'maf_hapmap_rel27', 'effect', 'stderr', 'pvalue']]

genotype_df.columns = ['rsID', 'chromosome', 'position', 'effect_allele', 'other_allele', 'maf_hapmap_rel27', 'effect', 'stderr', 'pvalue']

genotype_df['genotype'] = ['AA', 'AA', 'AG', 'GG', 'TT', 'TT', 'TC', 'CT', 'AA', 'GG', 'AA', 'AA', 'AG', 'GG', 'TT', 'TT', 'TC', 'CT', 'AA', 'GG']

# normalize snp ids in the genotype data
genotype_df['rsID'] = genotype_df['rsID'].str.strip().str.lower()

# find common snps from signifcant snps from genotype data
common_snps = set(gwas_summary['snp']).intersection(set(genotype_df['rsID']))
print(f"Number of common SNPs: {len(common_snps)}")

# filter gwas summary statistics and genotype data to include only common snps
filtered_gwas_summary = gwas_summary[gwas_summary['snp'].isin(common_snps)]
filtered_genotype_df = genotype_df[genotype_df['rsID'].isin(common_snps)]

print(f"Filtered GWAS Summary (first 5 rows):\n{filtered_gwas_summary.head()}")
print(f"Filtered Genotype Data (first 5 rows):\n{filtered_genotype_df.head()}")

# function to convert genotype to numeric value
def genotype_to_numeric(genotype):
    genotype_map = {'AA': 0, 'AG': 1, 'GA': 1, 'GG': 2, 'TT': 0, 'TC': 1, 'CT': 1, 'CC': 2}
    return genotype_map.get(genotype, np.nan)

# apply the conversion function to the genotype data
filtered_genotype_df['genotype_numeric'] = filtered_genotype_df['genotype'].apply(genotype_to_numeric)

# convert filtered genotype data to dictionary for prs calc
genotype_data_dict = pd.Series(filtered_genotype_df['genotype_numeric'].values, index=filtered_genotype_df['rsID']).to_dict()

GWAS Summary Statistics:
          snp  chr_build36  pos_build36 effect_allele other_allele  \
0   rs7517384            1     80017509             t            c   
1   rs1857095            1     80018608             t            c   
2  rs10495667            2     17686973             a            g   
3   rs7578326            2    226728897             a            g   
4  rs13405357            2    226730280             t            c   

   maf_hapmap_rel27  effect  stderr        pvalue  
0             0.033  -0.880   0.250  9.605000e-16  
1             0.034   0.880   0.250  5.398000e-16  
2             0.040  -0.900   0.180  3.826000e-09  
3             0.350   0.076   0.079  1.210000e-08  
4             0.350   0.100   0.090  2.145000e-08  
Number of common SNPs: 10
Filtered GWAS Summary (first 5 rows):
          snp  chr_build36  pos_build36 effect_allele other_allele  \
0   rs7517384            1     80017509             t            c   
1   rs1857095            1     8001860

In [57]:
# calculates the PRS 
def calculate_prs(gwas_summary, genotype_data_dict, p_value_col, snp_col='snp', effect_size_col='effect'):
    # filter for significant snps (p-value < 5e-8)
    significant_snps = gwas_summary[gwas_summary[p_value_col] < 5e-8]
    print(f"Number of significant SNPs: {len(significant_snps)}")
    
    # initialize prs score
    prs_score = 0
    
    # loop thru significant snps and calc the prs
    for index, snp in significant_snps.iterrows():
        snp_id = snp[snp_col]
        effect_size = snp[effect_size_col]
        
        # see if snp is in genotype data
        if snp_id in genotype_data_dict:
            genotype = genotype_data_dict[snp_id]
        else:
            print(f"SNP ID {snp_id} not found in genotype data. Skipping.")
            continue
        
        print(f"SNP ID: {snp_id}, Effect Size: {effect_size}, Genotype: {genotype}")
        
        # add the weighted effect size to the prs
        prs_score += effect_size * genotype
    
    return prs_score

# calculate the prs with the correct column names
prs = calculate_prs(gwas_summary, genotype_data_dict, p_value_col, snp_col=snp_col, effect_size_col=effect_size_col)
print(f"Polygenic Risk Score: {prs}")


Number of significant SNPs: 20
SNP ID: rs7517384, Effect Size: -0.88, Genotype: 0
SNP ID: rs1857095, Effect Size: 0.88, Genotype: 0
SNP ID: rs10495667, Effect Size: -0.9, Genotype: 1
SNP ID: rs7578326, Effect Size: 0.076, Genotype: 2
SNP ID: rs13405357, Effect Size: 0.1, Genotype: 0
SNP ID: rs6092841, Effect Size: -0.53, Genotype: 0
SNP ID: rs2828532, Effect Size: 1.1, Genotype: 1
SNP ID: rs2828535, Effect Size: 1.1, Genotype: 1
SNP ID: rs2828537, Effect Size: 1.1, Genotype: 0
SNP ID: rs10483182, Effect Size: -2.0, Genotype: 2
SNP ID: rs7517384, Effect Size: -0.88, Genotype: 0
SNP ID: rs1857095, Effect Size: 0.88, Genotype: 0
SNP ID: rs10495667, Effect Size: -0.9, Genotype: 1
SNP ID: rs7578326, Effect Size: 0.076, Genotype: 2
SNP ID: rs13405357, Effect Size: 0.1, Genotype: 0
SNP ID: rs6092841, Effect Size: -0.53, Genotype: 0
SNP ID: rs2828532, Effect Size: 1.1, Genotype: 1
SNP ID: rs2828535, Effect Size: 1.1, Genotype: 1
SNP ID: rs2828537, Effect Size: 1.1, Genotype: 0
SNP ID: rs104831